In [12]:
from xml.etree.ElementTree import parse
import numpy as np
#from Class_ID_Name import *
import os
import openslide

from PIL import Image

import glob

import cv2 as cv2

import torch
import torchvision
from torchvision import transforms

from torch.utils.data import Dataset, DataLoader

from shapely.geometry import Polygon
from skimage import draw

import matplotlib.pyplot as plt

import pickle

import argparse

import warnings

import logging

warnings.filterwarnings('ignore')

In [13]:
list_ = ['Page-{}'.format(11)]

for idx in list_:
    
    #log.info(" Start : {}".format(idx))

    image_path = sorted(glob.glob('../../../../Projects/Pathology/datasets/HE_IHC/Stomach/{}/*.svs'.format(idx)))
    xml_path = sorted(glob.glob('../../../../Projects/Pathology/datasets/HE_IHC/Stomach/{}/*.xml'.format(idx)))
    
    patient_name = [i.split('/')[-1].split('.')[0] for i in image_path]
    
    
    for num_ in range(len(patient_name)):
        
        patient = patient_name[num_]
        
        patient_svs_path = image_path[num_]
        patient_xml_path = xml_path[num_]

In [39]:
num = 1

with open('{}/{}/{}.pickle'.format('Dict1024', 'Page-1', all_patient_name[num]), 'rb') as ok:
    temp = pickle.load(ok)

In [45]:
import pandas as pd

In [110]:
info = pd.read_csv('../../../../Projects/Pathology/datasets/HE_IHC/Stomach/data_Stomach_0625.csv')

In [161]:
list_ = ['Page-1', 'Page-2', 'Page-3', 'Page-4', 'Page-5', 'Page-6', 'Page-7', 'Page-8', 'Page-9', 'Page-10']

In [162]:
for idx in list_:
    
    
    
    page_idx = [i.split('\\')[1] == idx for i in info.path.values]
    page_info = info[page_idx]
    page_patient_name = np.unique(page_info.id.values)
    
    temp = page_info.id.values
    print(temp.shape), print(np.unique(temp).shape) 
    print('\n')
    
    break

(72,)
(30,)




In [206]:
list_ = ['Page-1', 'Page-2', 'Page-3', 'Page-4', 'Page-5', 'Page-6', 'Page-7', 'Page-8', 'Page-9', 'Page-10']


for idx in list_:

    image_path = sorted(glob.glob('../../../../Projects/Pathology/datasets/HE_IHC/Stomach/{}/*.svs'.format(idx)))
    xml_path = sorted(glob.glob('../../../../Projects/Pathology/datasets/HE_IHC/Stomach/{}/*.xml'.format(idx)))
    
    patient_name = [i.split('/')[-1].split('.')[0] for i in image_path]
    
    
    for num in range(len(patient_name)):
        
        patient = patient_name[num]
        
        patient_svs_path = image_path[num]
        patient_xml_path = xml_path[num]
        
        
        temp = openslide.OpenSlide(patient_svs_path)


        x_s = []
        y_s = []
        min_x = []
        max_x = []
        min_y = []
        max_y = []

        points = []


        tree = parse(patient_xml_path)
        root = tree.getroot()  # 최상단 태그
        Annotations = root.findall('Annotation')

        for Annotation in Annotations:

            regions = Annotation.find("Regions").findall("Region")

            for region in regions:

                vertices = region.find("Vertices").findall("Vertex")
                region_X = []
                region_Y = []
                region_Points = []

                for vertex in vertices:
                    x = int(round(float(vertex.attrib['X'])))
                    y = int(round(float(vertex.attrib['Y'])))

                    region_X.append(x)
                    region_Y.append(y)
                    region_Points.append([x, y])

                x_s.append(region_X)
                y_s.append(region_Y)

                min_x.append(min(region_X))
                max_x.append(max(region_X))
                min_y.append(min(region_Y))
                max_y.append(max(region_Y))

                points.append(region_Points)


        downscale = 32
        binary_mask = np.zeros(shape=(int(temp.dimensions[1]/downscale), int(temp.dimensions[0]/downscale)))

        for i in range(len(points)):
            #region1_poly = Polygon(np.array(points[0]))
            #region2_poly = Polygon(np.array(points[1]))

            vertex_row_coords = np.array(points[i])[:,0]
            vertex_col_coords = np.array(points[i])[:,1]

            fill_row_coords, fill_col_coords = draw.polygon(np.array([int(i/downscale) for i in vertex_col_coords]),
                                                        np.array([int(i/downscale) for i in vertex_row_coords]),
                                                        binary_mask.shape)

            binary_mask[fill_row_coords, fill_col_coords] = 255



        ################### All Coords #################
        patch_size = 1024

        #a = np.ones(shape=(temp.dimensions[0], temp.dimensions[1]))

        width_ = int(temp.dimensions[0] / patch_size)
        height_ = int(temp.dimensions[1] / patch_size)

        all_coords = np.array([i[0] for i in np.ndenumerate(np.ones(shape=(width_, height_)))]) * 224






        ################### Hotspot Coords #################
        image_size = int( patch_size / downscale)
        height = int(binary_mask.shape[0] / image_size)
        width = int(binary_mask.shape[1] / image_size)


        temp1 = np.ones(shape=(width, height))
        coords = [i[0] for i in np.ndenumerate(temp1)]


        hotspot_ = []

        for a in range(height):
            for b in range(width):

                patch = binary_mask[a*image_size : (a+1)*image_size, b*image_size : (b+1)*image_size]

                if (patch == 255).sum() > 34:

                    y = [a*image_size]
                    x = [b*image_size]

                    hotspot_.append(x + y)
        hotspot_coords = np.array([i*downscale for i in np.array(hotspot_)])







        ################### Back Coords #################
        back_coords = []
        for i in range(len(all_coords)):

            patch = np.array(temp.read_region((all_coords[i][0], all_coords[i][1]), 0, (patch_size, patch_size)))[:, :, :3]

            if patch.mean() > 200:
                back_coords.append([all_coords[i][0], all_coords[i][1]])

        back_coords = np.array(back_coords)







        ################### No Hotspot Coords #################
        no_hotspot=[]
        for i in range(len(all_coords)):

            coords = all_coords[i].tolist()

            if np.isin(np.array([str(i.tolist()) for i in back_coords]), np.array(str(coords))).sum() == 1:
                continue


            elif np.isin(np.array([str(i.tolist()) for i in hotspot_coords]), np.array(str(coords))).sum() == 1:
                continue

            else:
                no_hotspot.append(coords)

                if len(no_hotspot) == 2000:
                    break





        Total_dict= {'{}'.format(patient_name): {}}

        back_label = np.zeros_like(back_coords)[:, 0].tolist()
        nohotspot_label = np.zeros_like(no_hotspot)[:, 0].tolist()
        hotspot_label = np.ones_like(hotspot_coords)[:, 0].tolist()

        back_size = 100
        nohotspot_size = 400
        hotspot_size = 500

        Total_dict['{}'.format(patient_name)]['slides_path'] = np.array(image_path)[np.array([i.split('/')[-1].split('.')[0] == patient_name for i in image_path])][0]
        Total_dict['{}'.format(patient_name)]['coords'] = back_coords.tolist()[:back_size] \
                                                        + no_hotspot[:nohotspot_size] \
                                                        + hotspot_coords.tolist()[:hotspot_size]
        Total_dict['{}'.format(patient_name)]['label'] = back_label[:back_size] + nohotspot_label[:nohotspot_size] + hotspot_label[:hotspot_size]





        result_path = 'Dict1024'
        if not os.path.exists('{}'.format(result_path)):
            os.mkdir('{}'.format(result_path))

        if not os.path.exists('{}/{}'.format(result_path, list_[num])):
            os.mkdir('{}/{}'.format(result_path, list_[num]))

        with open('{}/{}/{}.pickle'.format(result_path, list_[num], patient), 'wb') as ok:
            pickle.dump(Total_dict, ok)

        log.info('Back : {} , No Hotspot : {} Hotspot : {}'.format(len(back_coords.tolist()[:back_size]),
                                                                  len(no_hotspot[:nohotspot_size]),
                                                                  len(hotspot_coords.tolist()[:hotspot_size])))
        log.info('{} - {}   Complete'.format(list_[num], patient))
        log.info('\n')